In [1]:
import pandas as pd
import os
import json
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from urllib.parse import ParseResult

In [2]:
pickle_file = "hsbc_site_20250220.limited.pickle"
df:pd.DataFrame=pd.read_pickle(pickle_file)
df.head(1)

,catalog,files_links,url,title,content
0,accounts,[/content/dam/hsbc/hk/docs/premier/accounts-co...,https://www.hsbc.com.hk/accounts/,"Bank Accounts | Savings, Current, Time Deposit...","<!DOCTYPE html><html lang=""en-hk"" dir=""ltr"" it..."


In [3]:
documents=[]
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    content=row['content']
    soup = BeautifulSoup(content, 'html.parser')
    content_selectors = [
        'main'
        '.row transparent-bg intro-section hero-remove-height',
        'body'                      
    ]
    main_content = None
    for selector in content_selectors:
        elements = soup.select(selector)
        if elements:
            main_content = elements[0]
            break

    if not main_content:
        continue

    paragraphs = main_content.find_all('p')
    headings = main_content.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    lists = main_content.find_all(['ul', 'ol'])

    extracted_sections = []
    for heading in headings:
        extracted_sections.append({"type": "heading", "text": heading.get_text().strip()})
    for p in paragraphs:
        extracted_sections.append({"type": "paragraph", "text": p.get_text().strip()})
    for lst in lists:
        list_items = [li.get_text().strip() for li in lst.find_all('li')]
        extracted_sections.append({"type": "list", "items": list_items})

    cleaned_text_sections = []
    for section in extracted_sections:
        if section["type"] in ("paragraph", "heading"):
            text = section["text"]
            text = text.strip()
            text = re.sub(r'\s+', ' ', text)
            cleaned_text_sections.append({"type": section["type"], "text": text})
        elif section["type"] == "list":
            cleaned_items = []
            for item in section["items"]:
                item = item.strip()
                item = re.sub(r'\s+', ' ', item)
                cleaned_items.append(item)
            cleaned_text_sections.append({"type": "list", "items": cleaned_items})
    documents.append(cleaned_text_sections)
    pass

100%|██████████| 95/95 [00:07<00:00, 12.78it/s]


In [4]:
df['documents']=documents
df.to_pickle(pickle_file)